# SGLang을 활용한 LLM Agent Serving 튜토리얼

### 에이전트 아키텍처

본 튜토리얼에서 구현할 에이전트는 다음 요소들로 구성된다:
- **Pydantic**: 구조화된 출력 스키마 정의 및 검증
- **Function Calling & Tool**: SGLang의 구조화된 생성을 활용한 도구 호출
- **Memory**: RadixAttention 기반 효율적인 컨텍스트 관리
- **Validation**: 구조화된 출력의 스키마 검증
- **Recovery**: 생성 실패 시 재생성 및 대안 전략
- **Feedback**: 생성 품질 평가 및 개선

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
!pip install sglang pydantic requests anthropic jsonschema

In [1]:
import json
import sglang as sgl
from typing import List, Dict, Any, Optional, Callable, Union, Type
from datetime import datetime
from pydantic import BaseModel, Field, validator, ValidationError
import time
from enum import Enum

## 2. Pydantic 스키마 정의

In [2]:
class ToolCallDecision(str, Enum):
    """도구 호출 결정을 나타내는 열거형이다"""
    CALL_TOOL = "call_tool"
    RESPOND_DIRECTLY = "respond_directly"
    NEED_MORE_INFO = "need_more_info"

class ToolArgument(BaseModel):
    """도구 함수의 단일 인자를 표현하는 모델이다"""
    name: str = Field(description="인자 이름")
    value: Any = Field(description="인자 값")
    type_hint: str = Field(description="예상 타입")

class ToolCall(BaseModel):
    """구조화된 도구 호출을 표현하는 모델이다"""
    decision: ToolCallDecision = Field(description="도구 호출 여부 결정")
    tool_name: Optional[str] = Field(default=None, description="호출할 도구 이름")
    arguments: Optional[Dict[str, Any]] = Field(default=None, description="도구 인자")
    reasoning: str = Field(description="도구 호출 이유 또는 직접 응답 내용")
    confidence_score: float = Field(default=0.0, ge=0.0, le=1.0, description="결정 신뢰도")

class ToolSchema(BaseModel):
    """도구의 스키마를 정의하는 모델이다"""
    name: str = Field(description="도구 이름")
    description: str = Field(description="도구 기능 설명")
    parameters: Dict[str, Any] = Field(description="파라미터 스키마")
    return_type: str = Field(description="반환 타입")
    examples: List[Dict[str, Any]] = Field(default_factory=list, description="사용 예시")

class ToolDefinition(BaseModel):
    """실행 가능한 도구를 정의하는 모델이다"""
    schema: ToolSchema = Field(description="도구 스키마")
    callable: Callable = Field(description="실행할 함수")
    
    class Config:
        arbitrary_types_allowed = True

class Message(BaseModel):
    """대화 메시지를 표현하는 모델이다"""
    role: str = Field(description="메시지 역할")
    content: str = Field(description="메시지 내용")
    metadata: Dict[str, Any] = Field(default_factory=dict, description="추가 메타데이터")
    timestamp: datetime = Field(default_factory=datetime.now, description="생성 시간")
    token_count: Optional[int] = Field(default=None, description="토큰 수")
    
    @validator('role')
    def validate_role(cls, v):
        """역할이 유효한지 검증한다"""
        allowed = ['system', 'user', 'assistant', 'tool', 'observation']
        assert v in allowed, f"역할은 {allowed} 중 하나여야 한다"
        return v

class AgentResponse(BaseModel):
    """에이전트의 최종 응답을 표현하는 모델이다"""
    content: str = Field(description="응답 내용")
    tool_calls: List[Dict[str, Any]] = Field(default_factory=list, description="실행된 도구들")
    reasoning_steps: List[str] = Field(default_factory=list, description="추론 과정")
    total_tokens: int = Field(default=0, description="총 사용 토큰")
    cache_hit_rate: float = Field(default=0.0, description="캐시 적중률")
    latency_ms: float = Field(default=0.0, description="응답 시간")
    is_successful: bool = Field(default=True, description="성공 여부")
    error_message: Optional[str] = Field(default=None, description="오류 메시지")

class ValidationResult(BaseModel):
    """검증 결과를 표현하는 모델이다"""
    is_valid: bool = Field(description="검증 통과 여부")
    error_message: Optional[str] = Field(default=None, description="오류 메시지")
    validation_details: Dict[str, Any] = Field(default_factory=dict, description="상세 정보")
    suggested_fix: Optional[str] = Field(default=None, description="수정 제안")

/tmp/ipykernel_335141/777523221.py:29: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class ToolDefinition(BaseModel):
/tmp/ipykernel_335141/777523221.py:29: UserWarning: Field name "schema" in "ToolDefinition" shadows an attribute in parent "BaseModel"
  class ToolDefinition(BaseModel):
/tmp/ipykernel_335141/777523221.py:45: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('role')


## 3. Memory 시스템 구현

In [3]:
class ConversationNode:
    """대화 트리의 노드를 표현하는 클래스다"""
    
    def __init__(self, message: Message, parent: Optional['ConversationNode'] = None):
        """노드를 초기화한다"""
        self.message = message
        self.parent = parent
        self.children: List['ConversationNode'] = []
        self.node_id = f"node_{int(time.time() * 1000000)}"
    
    def add_child(self, message: Message) -> 'ConversationNode':
        """자식 노드를 추가한다"""
        child = ConversationNode(message, parent=self)
        self.children.append(child)
        return child
    
    def get_path_to_root(self) -> List[Message]:
        """루트까지의 경로를 메시지 리스트로 반환한다"""
        path = []
        current = self
        while current is not None:
            path.append(current.message)
            current = current.parent
        return list(reversed(path))

class RadixMemory:
    """RadixAttention 패턴을 활용한 트리 기반 메모리 시스템이다"""
    
    def __init__(self, max_depth: int = 50):
        """메모리 시스템을 초기화한다"""
        self.max_depth = max_depth
        self.root: Optional[ConversationNode] = None
        self.current_node: Optional[ConversationNode] = None
        self.all_nodes: Dict[str, ConversationNode] = {}
        self.cache_stats = {"hits": 0, "misses": 0}
    
    def initialize_with_system(self, system_message: str) -> None:
        """시스템 메시지로 루트를 초기화한다"""
        system_msg = Message(role="system", content=system_message)
        self.root = ConversationNode(system_msg)
        self.current_node = self.root
        self.all_nodes[self.root.node_id] = self.root
    
    def add_message(self, role: str, content: str, metadata: Dict[str, Any] = None) -> str:
        """현재 노드에 메시지를 추가하고 노드 ID를 반환한다"""
        message = Message(
            role=role,
            content=content,
            metadata=metadata or {}
        )
        
        if self.current_node is None:
            self.initialize_with_system("")
        
        new_node = self.current_node.add_child(message)
        self.all_nodes[new_node.node_id] = new_node
        self.current_node = new_node
        
        return new_node.node_id
    
    def get_conversation_history(self) -> List[Dict[str, str]]:
        """현재 노드까지의 대화 히스토리를 반환한다"""
        if self.current_node is None:
            return []
        
        messages = self.current_node.get_path_to_root()
        return [
            {"role": msg.role, "content": msg.content}
            for msg in messages
        ]
    
    def branch_from_node(self, node_id: str) -> bool:
        """특정 노드로부터 새로운 브랜치를 시작한다"""
        if node_id in self.all_nodes:
            self.current_node = self.all_nodes[node_id]
            self.cache_stats["hits"] += 1
            return True
        self.cache_stats["misses"] += 1
        return False
    
    def get_cache_hit_rate(self) -> float:
        """캐시 적중률을 계산한다"""
        total = self.cache_stats["hits"] + self.cache_stats["misses"]
        if total == 0:
            return 0.0
        return self.cache_stats["hits"] / total
    
    def get_statistics(self) -> Dict[str, Any]:
        """메모리 통계를 반환한다"""
        depth = 0
        if self.current_node:
            depth = len(self.current_node.get_path_to_root())
        
        return {
            "total_nodes": len(self.all_nodes),
            "current_depth": depth,
            "max_depth": self.max_depth,
            "cache_hit_rate": self.get_cache_hit_rate(),
            "cache_stats": self.cache_stats
        }
    
    def clear(self) -> None:
        """메모리를 초기화한다"""
        self.root = None
        self.current_node = None
        self.all_nodes.clear()
        self.cache_stats = {"hits": 0, "misses": 0}

## 4. Function Calling & Tool 시스템 구현

In [4]:
# 예제 도구 함수들을 정의한다

def analyze_sentiment(text: str, language: str = "ko") -> str:
    """텍스트의 감정을 분석한다"""
    sentiments = ["긍정적", "부정적", "중립적"]
    sentiment = sentiments[len(text) % 3]
    return f"텍스트 '{text[:30]}...'의 감정은 {sentiment}이다 (언어: {language})"

def retrieve_documents(query: str, top_k: int = 5, filter_criteria: str = None) -> str:
    """문서 데이터베이스에서 관련 문서를 검색한다"""
    result = f"'{query}' 검색 결과: {top_k}개의 관련 문서를 찾았다"
    if filter_criteria:
        result += f" (필터: {filter_criteria})"
    return result

def execute_code(code: str, language: str = "python") -> str:
    """코드를 실행하고 결과를 반환한다"""
    return f"{language} 코드 실행 완료: {len(code)}자의 코드가 성공적으로 실행되었다"

def generate_summary(text: str, max_length: int = 100, style: str = "concise") -> str:
    """텍스트를 요약한다"""
    summary = text[:max_length] + "..."
    return f"[{style} 스타일 요약] {summary}"

def translate_text(text: str, source_lang: str, target_lang: str) -> str:
    """텍스트를 번역한다"""
    return f"'{text}'를 {source_lang}에서 {target_lang}으로 번역했다"

In [5]:
class StructuredToolManager:
    """구조화된 도구 관리 시스템이다"""
    
    def __init__(self):
        """도구 매니저를 초기화한다"""
        self.tools: Dict[str, ToolDefinition] = {}
        self.execution_history: List[Dict[str, Any]] = []
        self.tool_performance: Dict[str, List[float]] = {}
    
    def register_tool(self, tool: ToolDefinition) -> None:
        """도구를 등록한다"""
        self.tools[tool.schema.name] = tool
        self.tool_performance[tool.schema.name] = []
    
    def get_tools_description(self) -> str:
        """모든 도구의 상세 설명을 반환한다"""
        descriptions = []
        for tool in self.tools.values():
            schema = tool.schema
            desc = f"\n도구: {schema.name}\n"
            desc += f"설명: {schema.description}\n"
            desc += f"파라미터: {json.dumps(schema.parameters, indent=2, ensure_ascii=False)}\n"
            desc += f"반환 타입: {schema.return_type}\n"
            if schema.examples:
                desc += f"사용 예시: {json.dumps(schema.examples, indent=2, ensure_ascii=False)}\n"
            descriptions.append(desc)
        return "\n".join(descriptions)
    
    def get_tool_schemas_json(self) -> List[Dict[str, Any]]:
        """JSON 스키마 형식으로 도구 정보를 반환한다"""
        return [tool.schema.dict() for tool in self.tools.values()]
    
    def execute_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """도구를 실행하고 결과를 반환한다"""
        tool = self.tools.get(tool_name)
        
        start_time = time.time()
        result = tool.callable(**arguments)
        execution_time = (time.time() - start_time) * 1000
        
        # 실행 기록 저장
        self.execution_history.append({
            "timestamp": datetime.now(),
            "tool_name": tool_name,
            "arguments": arguments,
            "result": result,
            "execution_time_ms": execution_time
        })
        
        # 성능 기록
        self.tool_performance[tool_name].append(execution_time)
        
        return result
    
    def get_tool(self, name: str) -> Optional[ToolDefinition]:
        """이름으로 도구를 검색한다"""
        return self.tools.get(name)
    
    def get_execution_statistics(self) -> Dict[str, Any]:
        """도구 실행 통계를 반환한다"""
        stats = {
            "total_executions": len(self.execution_history),
            "tools_used": {}
        }
        
        for tool_name, times in self.tool_performance.items():
            if times:
                stats["tools_used"][tool_name] = {
                    "call_count": len(times),
                    "avg_time_ms": sum(times) / len(times),
                    "min_time_ms": min(times),
                    "max_time_ms": max(times)
                }
        
        return stats
    
    def get_tool_recommendation(self, query: str) -> List[str]:
        """쿼리에 적합한 도구를 추천한다"""
        recommendations = []
        query_lower = query.lower()
        
        for tool in self.tools.values():
            if any(keyword in query_lower for keyword in 
                   tool.schema.description.lower().split()):
                recommendations.append(tool.schema.name)
        
        return recommendations

## 5. Validation 시스템 구현

In [6]:
class StructuredValidator:
    """구조화된 데이터의 검증을 담당하는 클래스다"""
    
    @staticmethod
    def validate_tool_call(tool_call: ToolCall, tool_schema: ToolSchema) -> ValidationResult:
        """도구 호출의 유효성을 검증한다"""
        details = {}
        
        # 결정 검증
        if tool_call.decision == ToolCallDecision.CALL_TOOL:
            if not tool_call.tool_name:
                return ValidationResult(
                    is_valid=False,
                    error_message="도구를 호출하기로 했지만 도구 이름이 없다",
                    validation_details=details,
                    suggested_fix="tool_name을 지정하거나 decision을 변경해야 한다"
                )
            
            if not tool_call.arguments:
                return ValidationResult(
                    is_valid=False,
                    error_message="도구를 호출하기로 했지만 인자가 없다",
                    validation_details=details,
                    suggested_fix="필요한 인자를 제공해야 한다"
                )
            
            # 필수 파라미터 검증
            required_params = tool_schema.parameters.get("required", [])
            details["required_params"] = required_params
            details["provided_params"] = list(tool_call.arguments.keys())
            
            for param in required_params:
                if param not in tool_call.arguments:
                    return ValidationResult(
                        is_valid=False,
                        error_message=f"필수 파라미터 '{param}'가 누락되었다",
                        validation_details=details,
                        suggested_fix=f"'{param}' 파라미터를 추가해야 한다"
                    )
        
        # 신뢰도 점수 검증
        details["confidence_score"] = tool_call.confidence_score
        if tool_call.confidence_score < 0.3:
            return ValidationResult(
                is_valid=False,
                error_message="신뢰도 점수가 너무 낮다",
                validation_details=details,
                suggested_fix="더 많은 정보를 수집하거나 사용자에게 확인을 요청해야 한다"
            )
        
        return ValidationResult(is_valid=True, validation_details=details)
    
    @staticmethod
    def validate_json_structure(data: Dict[str, Any], schema: Dict[str, Any]) -> ValidationResult:
        """JSON 데이터가 스키마와 일치하는지 검증한다"""
        from jsonschema import validate, ValidationError as JsonValidationError
        
        details = {"schema": schema, "data_keys": list(data.keys())}
        
        try:
            validate(instance=data, schema=schema)
            return ValidationResult(is_valid=True, validation_details=details)
        except JsonValidationError as e:
            return ValidationResult(
                is_valid=False,
                error_message=f"JSON 스키마 검증 실패: {str(e)}",
                validation_details=details,
                suggested_fix="데이터 구조를 스키마에 맞게 수정해야 한다"
            )
    
    @staticmethod
    def validate_message_content(content: str, max_length: int = 50000) -> ValidationResult:
        """메시지 내용의 유효성을 검증한다"""
        details = {"content_length": len(content)}
        
        if not content or not content.strip():
            return ValidationResult(
                is_valid=False,
                error_message="메시지 내용이 비어있다",
                validation_details=details,
                suggested_fix="유효한 메시지 내용을 제공해야 한다"
            )
        
        if len(content) > max_length:
            return ValidationResult(
                is_valid=False,
                error_message=f"메시지가 너무 길다 (최대 {max_length}자)",
                validation_details=details,
                suggested_fix="메시지를 분할하거나 요약해야 한다"
            )
        
        return ValidationResult(is_valid=True, validation_details=details)
    
    @staticmethod
    def validate_pydantic_model(model_class: Type[BaseModel], data: Dict[str, Any]) -> ValidationResult:
        """Pydantic 모델로 데이터를 검증한다"""
        try:
            model_class(**data)
            return ValidationResult(
                is_valid=True,
                validation_details={"model": model_class.__name__}
            )
        except ValidationError as e:
            return ValidationResult(
                is_valid=False,
                error_message=f"Pydantic 검증 실패: {str(e)}",
                validation_details={"model": model_class.__name__, "errors": e.errors()},
                suggested_fix="필드 값을 올바른 타입과 형식으로 수정해야 한다"
            )

## 6. Recovery 시스템 구현

In [7]:
class AdaptiveRecovery:
    """적응형 복구 전략을 제공하는 클래스다"""
    
    def __init__(self, max_retries: int = 3, backoff_factor: float = 1.5):
        """복구 시스템을 초기화한다"""
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.failure_log: List[Dict[str, Any]] = []
        self.recovery_strategies: Dict[str, List[str]] = {}
        self.retry_counts: Dict[str, int] = {}
    
    def log_failure(self, operation_id: str, error_type: str, 
                   error_message: str, context: Dict[str, Any]) -> None:
        """실패를 기록한다"""
        self.failure_log.append({
            "timestamp": datetime.now(),
            "operation_id": operation_id,
            "error_type": error_type,
            "error_message": error_message,
            "context": context
        })
    
    def should_retry(self, operation_id: str) -> bool:
        """재시도 가능 여부를 판단한다"""
        current_count = self.retry_counts.get(operation_id, 0)
        return current_count < self.max_retries
    
    def get_retry_delay(self, operation_id: str) -> float:
        """재시도 대기 시간을 계산한다"""
        retry_count = self.retry_counts.get(operation_id, 0)
        return self.backoff_factor ** retry_count
    
    def record_retry(self, operation_id: str) -> int:
        """재시도를 기록하고 현재 횟수를 반환한다"""
        self.retry_counts[operation_id] = self.retry_counts.get(operation_id, 0) + 1
        return self.retry_counts[operation_id]
    
    def reset_operation(self, operation_id: str) -> None:
        """작업의 재시도 카운터를 초기화한다"""
        if operation_id in self.retry_counts:
            del self.retry_counts[operation_id]
    
    def get_fallback_strategy(self, error_type: str) -> str:
        """오류 타입에 따른 폴백 전략을 반환한다"""
        strategies = {
            "generation_error": "더 간단한 프롬프트로 재시도한다",
            "validation_error": "스키마 제약을 완화하여 재생성한다",
            "tool_execution_error": "대체 도구를 사용하거나 수동 응답을 생성한다",
            "timeout_error": "생성 길이를 줄이고 재시도한다",
            "parsing_error": "출력 형식을 단순화하여 재생성한다"
        }
        return strategies.get(error_type, "기본 오류 처리를 수행한다")
    
    def generate_recovery_prompt(self, error_type: str, original_prompt: str) -> str:
        """오류 타입에 맞는 복구 프롬프트를 생성한다"""
        recovery_additions = {
            "generation_error": "\n\n이전 시도가 실패했다. 더 간단하고 명확한 응답을 생성하라.",
            "validation_error": "\n\n이전 응답이 형식 요구사항을 만족하지 못했다. 정확한 형식으로 다시 생성하라.",
            "tool_execution_error": "\n\n도구 실행에 실패했다. 도구 없이 직접 답변하라.",
            "timeout_error": "\n\n이전 응답이 너무 길었다. 50단어 이내로 간결하게 답변하라."
        }
        addition = recovery_additions.get(error_type, "\n\n오류가 발생했다. 다시 시도하라.")
        return original_prompt + addition
    
    def get_recovery_statistics(self) -> Dict[str, Any]:
        """복구 통계를 반환한다"""
        error_types = {}
        for failure in self.failure_log:
            error_type = failure["error_type"]
            error_types[error_type] = error_types.get(error_type, 0) + 1
        
        return {
            "total_failures": len(self.failure_log),
            "error_distribution": error_types,
            "active_retries": len(self.retry_counts),
            "operations_in_retry": list(self.retry_counts.keys())
        }
    
    def analyze_failure_patterns(self) -> List[str]:
        """실패 패턴을 분석하여 인사이트를 제공한다"""
        insights = []
        
        if len(self.failure_log) > 10:
            recent_failures = self.failure_log[-10:]
            error_types = [f["error_type"] for f in recent_failures]
            most_common = max(set(error_types), key=error_types.count)
            insights.append(f"최근 가장 빈번한 오류: {most_common}")
        
        if len(self.retry_counts) > 5:
            insights.append("많은 작업이 재시도 중이다. 시스템 부하를 확인해야 한다.")
        
        return insights

## 7. Feedback 시스템 구현

In [8]:
class QualityAssessment(BaseModel):
    """품질 평가 결과를 표현하는 모델이다"""
    accuracy_score: float = Field(ge=0.0, le=5.0, description="정확성 점수")
    coherence_score: float = Field(ge=0.0, le=5.0, description="일관성 점수")
    completeness_score: float = Field(ge=0.0, le=5.0, description="완전성 점수")
    overall_score: float = Field(ge=0.0, le=5.0, description="종합 점수")
    feedback_comment: str = Field(description="피드백 코멘트")

class FeedbackCollector:
    """에이전트 성능에 대한 피드백을 수집하고 분석하는 클래스다"""
    
    def __init__(self):
        """피드백 컬렉터를 초기화한다"""
        self.feedback_records: List[Dict[str, Any]] = []
        self.quality_assessments: List[QualityAssessment] = []
        self.performance_metrics: Dict[str, List[float]] = {
            "response_time": [],
            "token_usage": [],
            "cache_hit_rate": []
        }
    
    def add_feedback(self, query: str, response: str, 
                    rating: float, comment: str = "",
                    metadata: Dict[str, Any] = None) -> None:
        """사용자 피드백을 추가한다"""
        feedback = {
            "timestamp": datetime.now(),
            "query": query,
            "response": response,
            "rating": rating,
            "comment": comment,
            "metadata": metadata or {}
        }
        self.feedback_records.append(feedback)
    
    def add_quality_assessment(self, assessment: QualityAssessment) -> None:
        """품질 평가를 추가한다"""
        self.quality_assessments.append(assessment)
    
    def record_performance(self, response_time: float, 
                          token_usage: int, cache_hit_rate: float) -> None:
        """성능 메트릭을 기록한다"""
        self.performance_metrics["response_time"].append(response_time)
        self.performance_metrics["token_usage"].append(token_usage)
        self.performance_metrics["cache_hit_rate"].append(cache_hit_rate)
    
    def get_average_ratings(self) -> Dict[str, float]:
        """평균 평점을 계산한다"""
        if not self.feedback_records:
            return {"user_rating": 0.0}
        
        ratings = [f["rating"] for f in self.feedback_records]
        result = {"user_rating": sum(ratings) / len(ratings)}
        
        if self.quality_assessments:
            result["accuracy"] = sum(q.accuracy_score for q in self.quality_assessments) / len(self.quality_assessments)
            result["coherence"] = sum(q.coherence_score for q in self.quality_assessments) / len(self.quality_assessments)
            result["completeness"] = sum(q.completeness_score for q in self.quality_assessments) / len(self.quality_assessments)
            result["overall"] = sum(q.overall_score for q in self.quality_assessments) / len(self.quality_assessments)
        
        return result
    
    def get_performance_summary(self) -> Dict[str, Any]:
        """성능 요약을 반환한다"""
        summary = {}
        
        for metric, values in self.performance_metrics.items():
            if values:
                summary[metric] = {
                    "average": sum(values) / len(values),
                    "min": min(values),
                    "max": max(values),
                    "samples": len(values)
                }
        
        return summary
    
    def generate_improvement_recommendations(self) -> List[str]:
        """개선 권장사항을 생성한다"""
        recommendations = []
        
        # 평점 분석
        if self.feedback_records:
            avg_rating = sum(f["rating"] for f in self.feedback_records) / len(self.feedback_records)
            if avg_rating < 3.0:
                recommendations.append("평균 평점이 낮다. 응답 품질을 개선해야 한다.")
            elif avg_rating >= 4.5:
                recommendations.append("높은 평점을 유지하고 있다. 현재 전략을 지속한다.")
        
        # 성능 분석
        if self.performance_metrics["response_time"]:
            avg_time = sum(self.performance_metrics["response_time"]) / len(self.performance_metrics["response_time"])
            if avg_time > 5000:
                recommendations.append("평균 응답 시간이 5초를 초과한다. 캐싱 전략을 최적화해야 한다.")
        
        # 캐시 분석
        if self.performance_metrics["cache_hit_rate"]:
            avg_cache = sum(self.performance_metrics["cache_hit_rate"]) / len(self.performance_metrics["cache_hit_rate"])
            if avg_cache < 0.5:
                recommendations.append("캐시 적중률이 낮다. RadixAttention 활용을 개선해야 한다.")
            elif avg_cache > 0.8:
                recommendations.append("캐시 적중률이 우수하다. RadixAttention이 효과적으로 작동한다.")
        
        # 품질 분석
        if self.quality_assessments:
            avg_accuracy = sum(q.accuracy_score for q in self.quality_assessments) / len(self.quality_assessments)
            if avg_accuracy < 3.5:
                recommendations.append("정확성 점수가 낮다. 도구 활용과 검증 프로세스를 강화해야 한다.")
        
        return recommendations
    
    def get_feedback_summary(self) -> Dict[str, Any]:
        """전체 피드백 요약을 반환한다"""
        return {
            "total_feedback_count": len(self.feedback_records),
            "quality_assessment_count": len(self.quality_assessments),
            "average_ratings": self.get_average_ratings(),
            "performance_summary": self.get_performance_summary(),
            "recommendations": self.generate_improvement_recommendations()
        }

## 8. SGLang 에이전트 통합 구현

In [18]:
import sglang as sgl
from typing import List, Dict, Any
import time

class SGLangAgent:
    """SGLang을 사용하는 고급 에이전트 시스템이다"""
    
    def __init__(self, 
                 model_name: str = "openai/gpt-oss-20b",
                 host: str = "localhost",
                 port: int = 30000):
        """SGLang 에이전트를 초기화하고 모든 컴포넌트를 설정한다"""
        self.model_name = model_name
        self.backend_url = f"http://{host}:{port}"
        
        # 하위 시스템 초기화
        self.memory = RadixMemory()
        self.tool_manager = StructuredToolManager()
        self.validator = StructuredValidator()
        self.recovery = AdaptiveRecovery()
        self.feedback_collector = FeedbackCollector()
        
        # SGLang 런타임 설정
        sgl.set_default_backend(sgl.RuntimeEndpoint(self.backend_url))
        
        # 시스템 초기화
        self._initialize_system()
    
    def _initialize_system(self) -> None:
        """시스템 메시지와 기본 설정을 초기화한다"""
        system_message = f"""당신은 구조화된 출력을 생성하는 AI 어시스턴트다.
사용자의 질문을 분석하고, 필요한 경우 도구를 사용하여 정확한 답변을 제공한다.

사용 가능한 도구:
{self.tool_manager.get_tools_description()}

응답할 때는 다음 구조를 따라야 한다:
1. 도구 호출이 필요한지 판단한다
2. 필요하면 적절한 도구와 인자를 선택한다
3. 명확한 추론 과정을 제시한다
4. 신뢰도 점수를 평가한다"""
        
        self.memory.initialize_with_system(system_message)
    
    def generate_tool_decision(self, query: str) -> Dict[str, Any]:
        """도구 호출 결정을 생성하는 함수다"""
        
        # SGLang 함수를 클래스 외부에서 정의하거나, 여기서 동적으로 생성
        @sgl.function
        def _tool_decision_generator(s, user_query: str):
            """내부 SGLang 함수"""
            s += sgl.user(user_query)
            s += sgl.assistant(sgl.gen("reasoning", max_tokens=200))
            s += "\n\n도구 호출 결정: "
            s += sgl.gen(
                "decision",
                choices=["call_tool", "respond_directly", "need_more_info"],
                max_tokens=20
            )
        
        # 함수 실행
        state = _tool_decision_generator.run(user_query=query)
        
        return {
            "reasoning": state["reasoning"],
            "decision": state["decision"]
        }
    
    def _call_sglang(self, query: str, context: List[Dict[str, str]]) -> Dict[str, Any]:
        """SGLang을 호출하여 구조화된 응답을 생성한다"""
        
        # 컨텍스트를 프롬프트로 변환
        context_str = "\n".join([
            f"{msg['role']}: {msg['content']}" 
            for msg in context[-5:]  # 최근 5개만 사용
        ])
        
        full_prompt = f"""이전 대화:
{context_str}

현재 질문: {query}

위 질문을 분석하고 다음을 결정하시오:
1. 도구 호출이 필요한가?
2. 필요하다면 어떤 도구를 사용해야 하는가?
3. 도구 인자는 무엇인가?"""
        
        try:
            decision_result = self.generate_tool_decision(full_prompt)
            
            # 간단한 파싱 로직 (실제로는 더 정교하게 구현)
            return {
                "decision": decision_result.get("decision", "respond_directly"),
                "tool_name": "analyze_sentiment",  # 예시
                "arguments": {"text": query, "language": "ko"},
                "reasoning": decision_result.get("reasoning", ""),
                "confidence_score": 0.85
            }
        except Exception as e:
            # 폴백: 직접 응답
            return {
                "decision": "respond_directly",
                "tool_name": "",
                "arguments": {},
                "reasoning": f"질문에 직접 답변한다: {query}",
                "confidence_score": 0.7
            }
    
    def run(self, user_input: str, max_iterations: int = 10) -> AgentResponse:
        """사용자 입력을 처리하고 응답을 생성한다"""
        start_time = time.time()
        operation_id = f"op_{int(time.time() * 1000000)}"
        
        # 입력 검증
        validation = self.validator.validate_message_content(user_input)
        if not validation.is_valid:
            self.recovery.log_failure(
                operation_id,
                "validation_error",
                validation.error_message,
                {"input": user_input}
            )
            return AgentResponse(
                content=f"입력 검증 실패: {validation.error_message}",
                is_successful=False,
                error_message=validation.error_message
            )
        
        # 사용자 메시지 추가
        self.memory.add_message("user", user_input)
        
        tool_calls_made = []
        reasoning_steps = []
        total_tokens = 0
        
        # 반복적으로 도구 호출 및 실행
        for iteration in range(max_iterations):
            context = self.memory.get_conversation_history()
            
            # SGLang 호출하여 도구 결정 생성
            response_data = self._call_sglang(user_input, context)
            
            # ToolCall 객체로 변환
            tool_call = ToolCall(**response_data)
            reasoning_steps.append(tool_call.reasoning)
            
            # 도구 호출 결정에 따라 처리
            if tool_call.decision == ToolCallDecision.CALL_TOOL:
                tool = self.tool_manager.get_tool(tool_call.tool_name)
                
                # 도구 호출 검증
                validation = self.validator.validate_tool_call(tool_call, tool.schema)
                if not validation.is_valid:
                    if self.recovery.should_retry(operation_id):
                        self.recovery.record_retry(operation_id)
                        time.sleep(self.recovery.get_retry_delay(operation_id))
                        continue
                    return AgentResponse(
                        content=f"검증 실패: {validation.error_message}",
                        is_successful=False,
                        error_message=validation.error_message
                    )
                
                # 도구 실행
                tool_result = self.tool_manager.execute_tool(
                    tool_call.tool_name,
                    tool_call.arguments
                )
                
                tool_calls_made.append({
                    "tool": tool_call.tool_name,
                    "arguments": tool_call.arguments,
                    "result": tool_result,
                    "confidence": tool_call.confidence_score
                })
                
                # 도구 결과를 메모리에 추가
                self.memory.add_message(
                    "observation",
                    f"도구 '{tool_call.tool_name}' 실행 결과: {tool_result}"
                )
                
                # 최종 응답 생성을 위해 계속
                continue
            
            elif tool_call.decision == ToolCallDecision.RESPOND_DIRECTLY:
                # 직접 응답
                self.memory.add_message("assistant", tool_call.reasoning)
                
                latency_ms = (time.time() - start_time) * 1000
                cache_hit_rate = self.memory.get_cache_hit_rate()
                
                # 성능 기록
                self.feedback_collector.record_performance(
                    latency_ms,
                    total_tokens,
                    cache_hit_rate
                )
                
                self.recovery.reset_operation(operation_id)
                
                return AgentResponse(
                    content=tool_call.reasoning,
                    tool_calls=tool_calls_made,
                    reasoning_steps=reasoning_steps,
                    total_tokens=total_tokens,
                    cache_hit_rate=cache_hit_rate,
                    latency_ms=latency_ms,
                    is_successful=True
                )
            
            else:
                # 더 많은 정보 필요
                return AgentResponse(
                    content=f"추가 정보가 필요하다: {tool_call.reasoning}",
                    reasoning_steps=reasoning_steps,
                    is_successful=True
                )
        
        # 최대 반복 도달
        latency_ms = (time.time() - start_time) * 1000
        return AgentResponse(
            content="최대 반복 횟수에 도달했다. 현재까지의 결과를 반환한다.",
            tool_calls=tool_calls_made,
            reasoning_steps=reasoning_steps,
            total_tokens=total_tokens,
            latency_ms=latency_ms,
            is_successful=True
        )
    
    def add_feedback(self, query: str, response: str, rating: float, comment: str = "") -> None:
        """사용자 피드백을 추가한다"""
        self.feedback_collector.add_feedback(query, response, rating, comment)
    
    def add_quality_assessment(self, assessment: QualityAssessment) -> None:
        """품질 평가를 추가한다"""
        self.feedback_collector.add_quality_assessment(assessment)
    
    def get_agent_status(self) -> Dict[str, Any]:
        """에이전트의 전체 상태를 반환한다"""
        return {
            "memory_stats": self.memory.get_statistics(),
            "tool_stats": self.tool_manager.get_execution_statistics(),
            "recovery_stats": self.recovery.get_recovery_statistics(),
            "feedback_summary": self.feedback_collector.get_feedback_summary(),
            "failure_patterns": self.recovery.analyze_failure_patterns()
        }

## 9. 도구 등록 및 에이전트 초기화

In [19]:
# SGLang 에이전트 초기화
agent = SGLangAgent(
    model_name="openai/gpt-oss-20b",
    host="localhost",
    port=30000
)

# 감정 분석 도구 등록
sentiment_tool = ToolDefinition(
    schema=ToolSchema(
        name="analyze_sentiment",
        description="텍스트의 감정을 분석하여 긍정/부정/중립을 판단한다",
        parameters={
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "분석할 텍스트"
                },
                "language": {
                    "type": "string",
                    "description": "텍스트 언어",
                    "enum": ["ko", "en", "ja", "zh"]
                }
            },
            "required": ["text"]
        },
        return_type="string",
        examples=[
            {"text": "정말 좋은 날이다", "language": "ko", "result": "긍정적"}
        ]
    ),
    callable=analyze_sentiment
)
agent.tool_manager.register_tool(sentiment_tool)

# 문서 검색 도구 등록
retrieval_tool = ToolDefinition(
    schema=ToolSchema(
        name="retrieve_documents",
        description="문서 데이터베이스에서 관련 문서를 검색한다",
        parameters={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "검색 쿼리"
                },
                "top_k": {
                    "type": "integer",
                    "description": "반환할 문서 수",
                    "minimum": 1,
                    "maximum": 20
                },
                "filter_criteria": {
                    "type": "string",
                    "description": "필터 조건"
                }
            },
            "required": ["query"]
        },
        return_type="string"
    ),
    callable=retrieve_documents
)
agent.tool_manager.register_tool(retrieval_tool)

# 코드 실행 도구 등록
code_tool = ToolDefinition(
    schema=ToolSchema(
        name="execute_code",
        description="프로그래밍 코드를 실행하고 결과를 반환한다",
        parameters={
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "실행할 코드"
                },
                "language": {
                    "type": "string",
                    "description": "프로그래밍 언어",
                    "enum": ["python", "javascript", "java"]
                }
            },
            "required": ["code"]
        },
        return_type="string"
    ),
    callable=execute_code
)
agent.tool_manager.register_tool(code_tool)

# 요약 도구 등록
summary_tool = ToolDefinition(
    schema=ToolSchema(
        name="generate_summary",
        description="긴 텍스트를 지정된 길이와 스타일로 요약한다",
        parameters={
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "요약할 텍스트"
                },
                "max_length": {
                    "type": "integer",
                    "description": "최대 길이"
                },
                "style": {
                    "type": "string",
                    "description": "요약 스타일",
                    "enum": ["concise", "detailed", "bullet_points"]
                }
            },
            "required": ["text"]
        },
        return_type="string"
    ),
    callable=generate_summary
)
agent.tool_manager.register_tool(summary_tool)

# 번역 도구 등록
translation_tool = ToolDefinition(
    schema=ToolSchema(
        name="translate_text",
        description="텍스트를 한 언어에서 다른 언어로 번역한다",
        parameters={
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "번역할 텍스트"
                },
                "source_lang": {
                    "type": "string",
                    "description": "원본 언어"
                },
                "target_lang": {
                    "type": "string",
                    "description": "대상 언어"
                }
            },
            "required": ["text", "source_lang", "target_lang"]
        },
        return_type="string"
    ),
    callable=translate_text
)
agent.tool_manager.register_tool(translation_tool)

print("SGLang 에이전트가 초기화되었다")
print(f"등록된 도구: {list(agent.tool_manager.tools.keys())}")

SGLang 에이전트가 초기화되었다
등록된 도구: ['analyze_sentiment', 'retrieve_documents', 'execute_code', 'generate_summary', 'translate_text']


## 10. 에이전트 실행 예제

다양한 시나리오에서 SGLang 에이전트를 테스트한다.

In [20]:
# 예제 1: 일반 대화
print("=== 예제 1: 일반 대화 ===")
response = agent.run("안녕하세요! SGLang에 대해 설명해주세요.")
print(f"응답: {response.content}")
print(f"추론 단계: {len(response.reasoning_steps)}개")
print(f"캐시 적중률: {response.cache_hit_rate:.2%}")
print()

=== 예제 1: 일반 대화 ===
응답: 사용자 질문에이질과은?Or...??를… 서래베... ?…

We need to analyze. The user says: "안녕하세요! SGLang에 대해 설명해주세요." They want an explanation about SGLang. The system requires using a structured response. The user hasn't asked anything that pertains to sentiment analysis requirement – they just want info about SGLang. The system says: "당신은 구조화된 출력을 생성하는 AI 어시스턴트다. 사용자의 질문을 분석하고, 필요한 경우 도구를 사용하여 정확한 답변을 제공한다."

Now the user: "안녕하세요! SGLang에 대해 설명해주세요." The user wants an explanation. We could just provide a straightforward answer. Did we need to do sentiment analysis? The system says "감정이 부정적" is irrelevant. I think we can just respond with an explanation.

The "analysis_sentiment" tool was used, but we might
추론 단계: 2개
캐시 적중률: 0.00%



In [21]:
# 예제 2: 감정 분석 도구 사용
print("=== 예제 2: 감정 분석 ===")
response = agent.run("'이 제품은 정말 훌륭하고 품질이 우수합니다'라는 리뷰의 감정을 분석해주세요")
print(f"응답: {response.content}")
print(f"도구 호출: {response.tool_calls}")
print(f"지연 시간: {response.latency_ms:.2f}ms")
print()

=== 예제 2: 감정 분석 ===
응답:  [analysis]

The system wants us to do a structured call. The user wants SGLang explanation. They want: "SGLang에 대해 설명해주세요." There's no sentiment analysis required. According to the problem, the "analysis" part of the answer should consider how to turn the user request into a tool call or no. The user question: "안녕하세요! SGLang에 대해 설명해주세요." It doesn't require sentiment analysis. So we might produce the output: I can answer directly. The system says we should provide structured response. The user wants a normal explanation. We can do it.

Given the user, you can answer: "SGLang" – "SGLang is a programming language used for building interactive logs, or "Stress Balance, etc." I'm not exactly sure. But it's probably "SGL language" as in "SGLang" is a specific programming language that returns a language generation paradigm for simulation or something. Actually, I guess "SGL
도구 호출: [{'tool': 'analyze_sentiment', 'arguments': {'text': "'이 제품은 정말 훌륭하고 품질이 우수합니다'라는 리뷰의 감

In [22]:
# 예제 3: 복합 작업 - 검색 후 요약
print("=== 예제 3: 검색 및 요약 ===")
response = agent.run("'인공지능 윤리'에 대한 문서를 검색하고 간단히 요약해주세요")
print(f"응답: {response.content}")
print(f"실행된 도구 수: {len(response.tool_calls)}")
print(f"추론 과정:")
for i, step in enumerate(response.reasoning_steps, 1):
    print(f"  {i}. {step}")
print()

=== 예제 3: 검색 및 요약 ===
응답: 추가 정보가 필요하다:  [analysis] ...   [final]

We are to decide what to do. It's a request: J us search for '인공지능 윤리' doc and summarize. We need to consult Google Search. 
We do not need sentiment analysis. We use googlesearchtool. 
Then we gather search results. We might not actually produce the summary yet because we will call the tool, get evidence, then summarize. According to the instructions, respond. 
We must produce final answer only including the steps that we do done or the final. 
Yes the final outcome is the summary. 
But first we see instructions: "When you return a tool invocation the assistant must produce a response that contains a JSON object inside a fenced block with a single, top-level tool-call entry. The tool invocation should not contain extraneous keys." That is when we call googlesearch tool. That should be done. The user requests essential: search '인공지능 윤리' and summarize. 
Thus we
실행된 도구 수: 0
추론 과정:
  1.  [analysis] ...   [final]

We are to 

In [23]:
# 예제 4: 품질 평가 추가
print("=== 예제 4: 품질 평가 ===")
query = "'Hello World'를 한국어로 번역해주세요"
response = agent.run(query)

# 품질 평가 생성
assessment = QualityAssessment(
    accuracy_score=4.5,
    coherence_score=4.8,
    completeness_score=4.7,
    overall_score=4.6,
    feedback_comment="정확하고 완전한 번역이다"
)
agent.add_quality_assessment(assessment)

# 사용자 피드백 추가
agent.add_feedback(query, response.content, rating=4.5, comment="완벽한 번역이다")
print(f"품질 평가가 추가되었다")
print()

=== 예제 4: 품질 평가 ===
품질 평가가 추가되었다



## 11. RadixAttention 성능 테스트

In [24]:
print("=== RadixAttention 성능 테스트 ===")

# 연속적인 대화로 캐시 활용 테스트
queries = [
    "Python에 대해 알려주세요",
    "Python의 주요 특징은 무엇인가요?",
    "Python으로 할 수 있는 작업들을 알려주세요"
]

for i, query in enumerate(queries, 1):
    response = agent.run(query)
    print(f"\n질문 {i}: {query}")
    print(f"캐시 적중률: {response.cache_hit_rate:.2%}")
    print(f"지연 시간: {response.latency_ms:.2f}ms")

# 메모리 통계 확인
memory_stats = agent.memory.get_statistics()
print(f"\n메모리 통계:")
for key, value in memory_stats.items():
    print(f"  {key}: {value}")

=== RadixAttention 성능 테스트 ===

질문 1: Python에 대해 알려주세요
캐시 적중률: 0.00%
지연 시간: 0.00ms

질문 2: Python의 주요 특징은 무엇인가요?
캐시 적중률: 0.00%
지연 시간: 3411.81ms

질문 3: Python으로 할 수 있는 작업들을 알려주세요
캐시 적중률: 0.00%
지연 시간: 6143.35ms

메모리 통계:
  total_nodes: 24
  current_depth: 24
  max_depth: 50
  cache_hit_rate: 0.0
  cache_stats: {'hits': 0, 'misses': 0}


## 12. 종합 상태 및 분석

In [25]:
print("=== 에이전트 종합 상태 ===")
status = agent.get_agent_status()

print("\n[메모리 통계]")
for key, value in status["memory_stats"].items():
    print(f"  {key}: {value}")

print("\n[도구 실행 통계]")
tool_stats = status["tool_stats"]
print(f"  총 실행 횟수: {tool_stats.get('total_executions', 0)}")
if "tools_used" in tool_stats:
    for tool, stats in tool_stats["tools_used"].items():
        print(f"  {tool}:")
        for stat_key, stat_value in stats.items():
            print(f"    - {stat_key}: {stat_value}")

print("\n[복구 통계]")
for key, value in status["recovery_stats"].items():
    print(f"  {key}: {value}")

print("\n[피드백 요약]")
feedback = status["feedback_summary"]
print(f"  총 피드백 수: {feedback['total_feedback_count']}")
print(f"  품질 평가 수: {feedback['quality_assessment_count']}")

if "average_ratings" in feedback and feedback["average_ratings"]:
    print("\n  평균 평점:")
    for metric, value in feedback["average_ratings"].items():
        print(f"    {metric}: {value:.2f}")

print("\n  개선 권장사항:")
for rec in feedback.get("recommendations", []):
    print(f"    - {rec}")

print("\n[실패 패턴 분석]")
for pattern in status.get("failure_patterns", []):
    print(f"  - {pattern}")

=== 에이전트 종합 상태 ===

[메모리 통계]
  total_nodes: 24
  current_depth: 24
  max_depth: 50
  cache_hit_rate: 0.0
  cache_stats: {'hits': 0, 'misses': 0}

[도구 실행 통계]
  총 실행 횟수: 11
  analyze_sentiment:
    - call_count: 11
    - avg_time_ms: 0.010100277987393465
    - min_time_ms: 0.007152557373046875
    - max_time_ms: 0.01430511474609375

[복구 통계]
  total_failures: 0
  error_distribution: {}
  active_retries: 0
  operations_in_retry: []

[피드백 요약]
  총 피드백 수: 1
  품질 평가 수: 1

  평균 평점:
    user_rating: 4.50
    accuracy: 4.50
    coherence: 4.80
    completeness: 4.70
    overall: 4.60

  개선 권장사항:
    - 높은 평점을 유지하고 있다. 현재 전략을 지속한다.
    - 평균 응답 시간이 5초를 초과한다. 캐싱 전략을 최적화해야 한다.
    - 캐시 적중률이 낮다. RadixAttention 활용을 개선해야 한다.

[실패 패턴 분석]
